In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import itertools
from sklearn.metrics import confusion_matrix

from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import udf, col
from pyspark.sql.types import FloatType

In [2]:
df = spark.read.csv(
  path='/FileStore/tables/bank.csv', 
  sep=';',
  header=True, 
  inferSchema=True)
df.printSchema()

root
-- age: integer (nullable = true)
-- job: string (nullable = true)
-- marital: string (nullable = true)
-- education: string (nullable = true)
-- default: string (nullable = true)
-- balance: integer (nullable = true)
-- housing: string (nullable = true)
-- loan: string (nullable = true)
-- contact: string (nullable = true)
-- day: integer (nullable = true)
-- month: string (nullable = true)
-- duration: integer (nullable = true)
-- campaign: integer (nullable = true)
-- pdays: integer (nullable = true)
-- previous: integer (nullable = true)
-- poutcome: string (nullable = true)
-- y: string (nullable = true)

In [3]:
pd.DataFrame(df.take(3), columns=df.columns).transpose()

,0,1,2
age,30,33,35
job,unemployed,services,management
marital,married,married,single
education,primary,secondary,tertiary
default,no,no,no
balance,1787,4789,1350
housing,no,yes,yes
loan,no,yes,no
contact,cellular,cellular,cellular
day,19,11,16


In [5]:
# Unbalanced Dataset

targetColName = 'y'
df.groupBy(targetColName).count().orderBy('count').show()

+---+-----+
 y|count|
+---+-----+
yes| 521|
 no| 4000|
+---+-----+

In [6]:
# Summarize numeric features

numColNames = [colName for colName, dtype in df.dtypes if dtype in ('int', 'float')]
catgColNames = [colName for colName, dtype in df.dtypes if dtype in ('string', 'bool')]

In [7]:
df.select(numColNames).describe().show()

+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
summary| age| balance| day| duration| campaign| pdays| previous|
+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
 count| 4521| 4521| 4521| 4521| 4521| 4521| 4521|
 mean| 41.17009511170095|1422.6578190665782|15.915284229152842|263.96129174961294| 2.793629727936297|39.766644547666445|0.5425790754257908|
 stddev|10.576210958711263|3009.6381424673395| 8.247667327229934|259.85663262468216|3.1098066601885823|100.12112444301656|1.6935623506071211|
 min| 19| -3313| 1| 4| 1| -1| 0|
 max| 87| 71188| 31| 3025| 50| 871| 25|
+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+

In [8]:
catgColNames.remove('y')

stages = list()
for catgColName in catgColNames:
    stringIndexer = StringIndexer(inputCol = catgColName, outputCol = catgColName + 'Index')
    oneHotEncoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[catgColName + "classVec"])
    stages.extend([stringIndexer, oneHotEncoder])
    
yStringIndexer = StringIndexer(inputCol = 'y', outputCol = 'label')
stages.append(yStringIndexer)

assemblerInputs = [c + "classVec" for c in catgColNames] + numColNames
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages.append(assembler)

In [9]:
pipeline = Pipeline(stages=stages)
pipelineModel = pipeline.fit(df)
df = pipelineModel.transform(df)
df.printSchema()

root
-- age: integer (nullable = true)
-- job: string (nullable = true)
-- marital: string (nullable = true)
-- education: string (nullable = true)
-- default: string (nullable = true)
-- balance: integer (nullable = true)
-- housing: string (nullable = true)
-- loan: string (nullable = true)
-- contact: string (nullable = true)
-- day: integer (nullable = true)
-- month: string (nullable = true)
-- duration: integer (nullable = true)
-- campaign: integer (nullable = true)
-- pdays: integer (nullable = true)
-- previous: integer (nullable = true)
-- poutcome: string (nullable = true)
-- y: string (nullable = true)
-- jobIndex: double (nullable = false)
-- jobclassVec: vector (nullable = true)
-- maritalIndex: double (nullable = false)
-- maritalclassVec: vector (nullable = true)
-- educationIndex: double (nullable = false)
-- educationclassVec: vector (nullable = true)
-- defaultIndex: double (nullable = false)
-- defaultclassVec: vector (nullable = true)
-- housingIndex: double (nullable = false)
-- housingclassVec: vector (nullable = true)
-- loanIndex: double (nullable = false)
-- loanclassVec: vector (nullable = true)
-- contactIndex: double (nullable = false)
-- contactclassVec: vector (nullable = true)
-- monthIndex: double (nullable = false)
-- monthclassVec: vector (nullable = true)
-- poutcomeIndex: double (nullable = false)
-- poutcomeclassVec: vector (nullable = true)
-- label: double (nullable = false)
-- features: vector (nullable = true)

Logistic Regression

In [11]:
train, test = df.randomSplit([0.8, 0.2], seed = 2018)

In [12]:
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
lrModel = lr.fit(train)
predictions = lrModel.transform(test)

In [13]:
# Notice that predictions is a SparkDf which has all the columns of test and three additional columns (rawPrediction, probablity and prediction)
predictions.printSchema()

root
-- age: integer (nullable = true)
-- job: string (nullable = true)
-- marital: string (nullable = true)
-- education: string (nullable = true)
-- default: string (nullable = true)
-- balance: integer (nullable = true)
-- housing: string (nullable = true)
-- loan: string (nullable = true)
-- contact: string (nullable = true)
-- day: integer (nullable = true)
-- month: string (nullable = true)
-- duration: integer (nullable = true)
-- campaign: integer (nullable = true)
-- pdays: integer (nullable = true)
-- previous: integer (nullable = true)
-- poutcome: string (nullable = true)
-- y: string (nullable = true)
-- jobIndex: double (nullable = false)
-- jobclassVec: vector (nullable = true)
-- maritalIndex: double (nullable = false)
-- maritalclassVec: vector (nullable = true)
-- educationIndex: double (nullable = false)
-- educationclassVec: vector (nullable = true)
-- defaultIndex: double (nullable = false)
-- defaultclassVec: vector (nullable = true)
-- housingIndex: double (nullable = false)
-- housingclassVec: vector (nullable = true)
-- loanIndex: double (nullable = false)
-- loanclassVec: vector (nullable = true)
-- contactIndex: double (nullable = false)
-- contactclassVec: vector (nullable = true)
-- monthIndex: double (nullable = false)
-- monthclassVec: vector (nullable = true)
-- poutcomeIndex: double (nullable = false)
-- poutcomeclassVec: vector (nullable = true)
-- label: double (nullable = false)
-- features: vector (nullable = true)
-- rawPrediction: vector (nullable = true)
-- probability: vector (nullable = true)
-- prediction: double (nullable = false)

In [14]:
print(predictions.select('rawPrediction').take(1)[0])
print(len(predictions.select('rawPrediction').take(1)))
print(predictions.select('rawPrediction', 'probability', 'prediction').take(1)[0])

Row(rawPrediction=DenseVector([1.8428, -1.8428]))
1
Row(rawPrediction=DenseVector([1.8428, -1.8428]), probability=DenseVector([0.8633, 0.1367]), prediction=0.0)

In [15]:
predictions.select('rawPrediction', 'probability', 'prediction').show(3)

+--------------------+--------------------+----------+
 rawPrediction| probability|prediction|
+--------------------+--------------------+----------+
[1.84280120473312...|[0.86327966426825...| 0.0|
[4.21024313148777...|[0.98537432619654...| 0.0|
[3.52697539694562...|[0.97144563232686...| 0.0|
+--------------------+--------------------+----------+
only showing top 3 rows

In [16]:
beta = np.sort(lrModel.coefficients)
beta

Out[ 14 ]: array([ -1.44392372e+00, -1.20140185e+00, -8.59480344e-01,
 -8.50525680e-01, -7.03604676e-01, -5.60904375e-01,
 -4.57565818e-01, -4.45540251e-01, -4.30599213e-01,
 -4.25719196e-01, -4.08511474e-01, -3.33123329e-01,
 -2.85212498e-01, -2.69378414e-01, -2.53344795e-01,
 -2.12211666e-01, -1.90377971e-01, -1.21348756e-01,
 -1.10968817e-01, -6.34509999e-02, -3.64954470e-02,
 -2.63057360e-02, -1.28601083e-05, 5.71547756e-04,
 5.97455576e-04, 4.45346933e-03, 4.46465254e-03,
 2.82606412e-02, 9.21317092e-02, 1.35028050e-01,
 1.50554536e-01, 1.87304681e-01, 2.98586977e-01,
 3.88582414e-01, 4.18571022e-01, 5.18178617e-01,
 6.02670100e-01, 6.20833004e-01, 6.64661019e-01,
 1.06443302e+00, 1.49970421e+00, 2.04681338e+00])

In [17]:
trainingSummary = lrModel.summary
trainingSummary.roc.show(3)


+--------------------+-------------------+
 FPR| TPR|
+--------------------+-------------------+
 0.0| 0.0|
0.004523424878836834|0.04805491990846682|
0.008723747980613893|0.09839816933638444|
+--------------------+-------------------+
only showing top 3 rows

In [18]:
trainingSummary.pr.show(3)

+-------------------+------------------+
 recall| precision|
+-------------------+------------------+
 0.0| 0.6|
0.04805491990846682| 0.6|
0.09839816933638444|0.6142857142857143|
+-------------------+------------------+
only showing top 3 rows

In [19]:
# Evaluator API:  https://people.eecs.berkeley.edu/~jegonzal/pyspark/_modules/pyspark/ml/evaluation.html

evaluator = BinaryClassificationEvaluator()
print('Test Area Under ROC', evaluator.evaluate(predictions))

Test Area Under ROC 0.88774006840305

In [20]:
# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
    .addGrid(lr.regParam, [0.01, 0.5, 2.0])
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
    .addGrid(lr.maxIter, [1, 5, 10])
    .build()
)

# Create 3-fold CrossValidator
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=3)
cvModel = cv.fit(train)


/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [21]:
# Attributes of model can be obtained from: https://spark.apache.org/docs/2.2.0/api/python/pyspark.ml.html#pyspark.ml.classification.LogisticRegressionModel

cvModel.bestModel.coefficients

Out[ 47 ]: DenseVector([0.057, -0.2749, 0.0, 0.0, 0.0, 0.5308, 0.0, -0.0213, 0.0, 0.0, 0.4643, -0.1706, 0.0, -0.0383, 0.0904, 0.0, -0.7278, -0.3967, 0.2989, 0.2769, -0.4517, -0.2264, -0.1638, 0.0, 0.0761, -0.0572, 0.2899, 0.0582, -0.0068, 1.637, 1.1534, 2.2168, -0.5934, -0.4507, 0.0, 0.0005, 0.0, 0.0, 0.0035, -0.0311, 0.0003, 0.0322])

In [22]:
predictions = cvModel.transform(test)

evaluator = BinaryClassificationEvaluator()
print('Test Area Under ROC', evaluator.evaluate(predictions))

Test Area Under ROC 0.8739147592738765

In [23]:
train.select('y', 'label').show(10)

+---+-----+
 y|label|
+---+-----+
yes| 1.0|
yes| 1.0|
 no| 0.0|
 no| 0.0|
 no| 0.0|
yes| 1.0|
 no| 0.0|
 no| 0.0|
 no| 0.0|
 no| 0.0|
+---+-----+
only showing top 10 rows

**Adding a new column with weights to account for the imbalance in the labels of the dataset**

Reference: https://stackoverflow.com/questions/33372838/dealing-with-unbalanced-datasets-in-spark-mllib

In [26]:
df.groupBy('label').count().show()

nZeros = df.filter(df['label'] == 0).count()
nRows = df.count()
balancingRatioZeros = (nRows - nZeros) / nRows
balancingRatioOnes = 1 - balancingRatioZeros

def calculateWeights(label):
  if (label == 1.0):
    return balancingRatioOnes
  else:
    return balancingRatioZeros
  
calculateWeightsUdf = udf(calculateWeights, FloatType()) 

df = df.withColumn('labelWeight', calculateWeightsUdf(col("label")))

df.select('label', 'labelWeight').show()

+-----+-----+
label|count|
+-----+-----+
 0.0| 4000|
 1.0| 521|
+-----+-----+

+-----+-----------+
label|labelWeight|
+-----+-----------+
 0.0| 0.11523999|
 0.0| 0.11523999|
 0.0| 0.11523999|
 0.0| 0.11523999|
 0.0| 0.11523999|
 0.0| 0.11523999|
 0.0| 0.11523999|
 0.0| 0.11523999|
 0.0| 0.11523999|
 0.0| 0.11523999|
 0.0| 0.11523999|
 0.0| 0.11523999|
 0.0| 0.11523999|
 1.0| 0.88476|
 0.0| 0.11523999|
 0.0| 0.11523999|
 0.0| 0.11523999|
 0.0| 0.11523999|
 0.0| 0.11523999|
 0.0| 0.11523999|
+-----+-----------+
only showing top 20 rows

In [27]:
train, test = df.randomSplit([0.7, 0.2], seed = 2018)

evaluator = BinaryClassificationEvaluator()
lr = LogisticRegression(
  featuresCol='features',
  labelCol ='label', 
  weightCol='labelWeight',
  maxIter=10)

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
    .addGrid(lr.regParam, [0.01, 0.5, 2.0])
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
    .addGrid(lr.maxIter, [1, 5, 10])
    .build()
)

# Create 3-fold CrossValidator
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=3)
cvModel = cv.fit(train)
predictions = cvModel.transform(test)

print('Test Area Under ROC', evaluator.evaluate(predictions))

In [28]:
predictions.select('label', 'prediction').show(8)

+-----+----------+
label|prediction|
+-----+----------+
 0.0| 1.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 1.0|
 0.0| 1.0|
 0.0| 0.0|
 0.0| 0.0|
+-----+----------+
only showing top 8 rows

In [29]:
yTrue = predictions.select('label').toPandas()
yPred = predictions.select('prediction').toPandas()
confMatrix = confusion_matrix(y_true=yTrue, y_pred=yPred)

In [30]:
confMatrix

Out[ 101 ]: array([[747, 158],
 [ 18, 66]])

In [31]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
  
plot_confusion_matrix(confMatrix, classes=['0', '1'])  

Confusion matrix, without normalization
[[747 158]
 [ 18 66]]

Descision Tree

In [33]:
evaluator = BinaryClassificationEvaluator()
dt = DecisionTreeClassifier(labelCol='label', featuresCol='features')

paramGrid = (ParamGridBuilder()
             .addGrid(dt.maxDepth, [1, 2, 6, 10])
             .addGrid(dt.maxBins, [20, 40, 80])
             .build())

# Create 3-fold CrossValidator
cv = CrossValidator(estimator=dt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=3)
cvModel = cv.fit(train)

In [34]:
predictions = cvModel.transform(test)

print('Test Area Under ROC', evaluator.evaluate(predictions))

Test Area Under ROC 0.46385135135135136

Area under ROC is unusually low. Calculate AUC using sklearn

In [36]:
yTrue = predictions.select('label').toPandas()
yPred = predictions.select('prediction').toPandas()

def print_conf_matrix(y_test, y_pred): 
    confusion_df = pd.DataFrame(
        data=confusion_matrix(y_test, y_pred), 
        columns=["Predicted Class " + str(class_name) for class_name in [0,1]], 
        index = ["Class " + str(class_name) for class_name in [0,1]]
    )
    print(confusion_df)

print_conf_matrix(yTrue, yPred)

Predicted Class 0 Predicted Class 1
Class 0 772 48
Class 1 44 30

I get different area-under ROC when using sklearn's metrics.auc

In [38]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from sklearn.metrics import roc_curve, auc

secondElement = udf(lambda v:float(v[1]), FloatType())
predictions = predictions.withColumn('class1Prob', secondElement('probability'))

yProb = predictions.select('class1Prob').toPandas()

fpr, tpr, threshold = roc_curve(yTrue, yProb)
auc = auc(fpr, tpr)
auc

Out[ 199 ]: 0.7075230718523402

Random Forest Classifier

In [40]:
evaluator = BinaryClassificationEvaluator()
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

paramGrid = (ParamGridBuilder()
    .addGrid(rf.maxDepth, [2, 4, 6])
    .addGrid(rf.maxBins, [20, 60])
    .addGrid(rf.numTrees, [5, 20])
    .build()
)
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=3)
cvModel = cv.fit(train)
predictions = cvModel.transform(test)

print('Test Area Under ROC', evaluator.evaluate(predictions))

Test Area Under ROC 0.8857366512854292

Gradient Boosting

In [42]:
evaluator = BinaryClassificationEvaluator()
gbt = GBTClassifier(maxIter=10)

paramGrid = (ParamGridBuilder()
    .addGrid(gbt.maxDepth, [2, 4, 6])
    .addGrid(gbt.maxBins, [20, 60])
    .addGrid(gbt.maxIter, [10, 20])
    .build()
)

cv = CrossValidator(estimator=gbt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=3)
cvModel = cv.fit(train)
predictions = cvModel.transform(test)

print('Test Area Under ROC', evaluator.evaluate(predictions))

Test Area Under ROC 0.8833717864205664